In [56]:
import os
import cv2
import base64
import requests
import pickle

In [57]:
path = "image/train" # Cars Dataset/train & Cats Dataset/test
list_x = []
list_y = []

In [58]:
def imgtovec(img):
    try:
        resized_img = cv2.resize(img, (128, 128), cv2.INTER_AREA)
        v, buffer = cv2.imencode(".jpg", resized_img)
        img_str = base64.b64encode(buffer).decode('utf-8')
        image_data_string = "data:image/jpeg;base64," + img_str

        url = "http://localhost:8000/api/genhog"
        params = {"img": image_data_string}

        response = requests.get(url, json=params)

        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"เรียก API ไม่สำเร็จ API CODE : {response.status_code}"}
    except Exception as ex:
        return {"error": f"เกิดข้อผิดพลาด: {str(ex)}"}

In [59]:
for sub in os.listdir(path):
    for fn in os.listdir(os.path.join(path, sub)):
        path_file_img = os.path.join(path, sub, fn)
        readImage = cv2.imread(path_file_img, cv2.IMREAD_GRAYSCALE)
        list_x.append(readImage)
        list_y.append(sub)

In [60]:
hogvectors = []
for i in range(len(list_x)):
    res = imgtovec(list_x[i])
    vec = list(res["HOG"])
    vec.append(list_y[i])
    hogvectors.append(vec)


In [61]:
write_path = "hogvectors_train.pkl"
pickle.dump(hogvectors, open(write_path, "wb"))
print("data preparation is done")

data preparation is done
